In [14]:
class Expression:
    def __call__(self, **context):
        pass

    def d(self, var):
        pass    

    def __add__(self, other):
        return Sum(self, other)
    
    def __mul__(self, other):
        return Product(self, other)
    
    def __truediv__(self, other):
        return Fraction(self, other)
    
    def __sub__(self, other):
        return self + Const(-1)*other
    

In [15]:
class Const(Expression):
    def __init__(self, const):
        self.const = const

    def __call__(self, **context):
        return self.const

    def d(self, var):
        return Const(0)

In [16]:
class Variable(Expression):
    def __init__(self, var):
        self.var = var

    def __call__(self, **context):
        return context[self.var]

    def d(self, var):
        if self.var == var.var:
            return Const(1)
        return Const(0)

In [17]:
V = Variable
C = Const

In [18]:
V("x").d(V("y"))()

0

In [19]:
V("x")(x=42)

42

In [20]:
class BinOP(Expression):
    def __init__(self, expr1, expr2):
        self.expr1 = expr1
        self.expr2 = expr2

In [21]:
class Sum(BinOP):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, var):
        return self.expr1.d(var) + self.expr2.d(var)
    

In [22]:
Sum(V("x").d(V("x")), V("y"))(x=2, y=3)

4

In [23]:
class Product(BinOP):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)

    def d(self, var):
        u = self.expr1
        du = self.expr1.d(var)
        v = self.expr2
        dv = self.expr2.d(var)
        return u*dv + v*du

In [26]:
(C(5) * V("x") * V("x") + C(3) * V("x") +C(2))(x=3)

56

In [27]:
class Fraction(BinOP):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)

    def d(self, var):
        u = self.expr1
        du = self.expr1.d(var)
        v = self.expr2
        dv = self.expr2.d(var)
        return (u*dv - v*du) / v**2


In [28]:
def newton_raphson(f, x0, eps):
    x = V("x")
    g = x - f / f.d(x)
    prev = x0
    xn = x0 + 2*eps
    while (xn-prev) > eps:
        prev = xn
        xn = g(x=xn)
    return xn


In [35]:
x = V("x")
f = (C(-5) *x*x + C(3) * x +C(2))
newton_raphson(f, 0.5, 1e-4)

1.1470007299787688